In [ ]:
from flask import Flask, render_template,request
from docx import Document
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
import random
import sys
import pandas as pd
import os
from win32com import client

import pythoncom

pythoncom.CoInitialize()

app = Flask(__name__)
#app.secret_key = 'development key'


@app.route('/',methods=['GET', 'POST'])
def contact():
    document = Document()
    df = pd.read_csv('objective.csv')
    pythoncom.CoInitialize()

    document.add_heading('Resume', 0)
    error = None
    qualifications = ['Btech','Mtech', 'BBA']
    sectors = ['IT','Mechanical', 'Automobile','Electrical']
    experience_level = ['Entry','Experienced']
    year_passing = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
    experience = ['0 - 1 yrs','2 - 4 yrs','5 - 7 yrs','8 - 10 yrs','10 - 12 yrs']
    if request.method =='POST':
        first_name = request.form['first_name']
        if first_name == '':
            error = 'Please fill first name'
        last_name = request.form['last_name']
        email = request.form['email']
        gender = request.form['gender']
        job_location = request.form['job_location']
        mobile_number = request.form['contact_number']
        my_skills = request.form.getlist('skills')
        degree1 = request.form['degree1']
        college1 = request.form['college1']
        cgpa1 = request.form['cgpa1']
        degree2 = request.form['degree2']
        college2 = request.form['college2']
        cgpa2 = request.form['cgpa2']
        degree3 = request.form['degree3']
        college3 = request.form['college3']
        cgpa3 = request.form['cgpa3']
        my_qualification = request.form.get('qualifications')
        my_passing = request.form.get('year_passing')
        my_experience = request.form.get('experience')
        my_gender = request.form['gender']
        #my_objective = request.form['message']
        my_location = request.form['job_location']
        github = request.form['github']
        linkedin = request.form['linkedin']
        my_industry = request.form.get('sectors')
        my_experience_level = request.form.get('experience_level')
        
        
        
        k = df[(df['Level']==my_experience_level) & (df['Career']==my_industry)]['Message']
        rand = random.randint(0,len(k))
        p = k.index[rand]
        
        my_objective = df[(df['Level']==my_experience_level) & (df['Career']==my_industry)]['Message'][p]
        
        
        p = document.add_paragraph('Name: '+first_name+' ' + last_name)
        p = document.add_paragraph('Gender: '+gender)
        p = document.add_paragraph('Contact: '+mobile_number)
        p = document.add_paragraph('Github Profile: '+ github)
        p = document.add_paragraph('LinkedIn Profile: '+linkedin)
        p = document.add_paragraph('My Objective: ' +my_objective)
        p = document.add_heading('Education', level=1)
        
        records = ((degree1, college1, cgpa1),(degree2 ,college2,cgpa2),(degree3 ,college3,cgpa3))
        
        
        table = document.add_table(rows=1, cols=3)
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'Degree'
        hdr_cells[1].text = 'College'
        hdr_cells[2].text = 'CGPA'
        for qty, id, desc in records:
            row_cells = table.add_row().cells
            row_cells[0].text = str(qty)
            row_cells[1].text = id
            row_cells[2].text = desc
        
        p = document.add_paragraph('Skills: ')
        for i in range(len(my_skills)):
            p = document.add_paragraph(my_skills[i], style='List Bullet')
        p = document.add_paragraph('Experience: '+my_experience)
        p = document.add_paragraph('Prefered Job Location: '+my_location)
        document.save('Resume.docx',) 
 
        
        fromaddr = YOUR EMAIL ID
        toaddr = email

        # instance of MIMEMultipart 
        msg = MIMEMultipart() 

        # storing the senders email address   
        msg['From'] = fromaddr 

        # storing the receivers email address  
        msg['To'] = toaddr 

        # storing the subject  
        msg['Subject'] = "Resume"

        # string to store the body of the mail 
        body = "Thanks for using resume builder please find the attached resume. "

        # attach the body with the msg instance 
        msg.attach(MIMEText(body, 'plain')) 

        # open the file to be sent  
        filename = "Resume.docx"
        attachment = open("Resume.docx", "rb") 

        # instance of MIMEBase and named as p 
        p = MIMEBase('application', 'octet-stream') 

        # To change the payload into encoded form 
        p.set_payload((attachment).read()) 
        attachment.close()
        # encode into base64 
        encoders.encode_base64(p) 

        p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 

        # attach the instance 'p' to instance 'msg' 
        msg.attach(p) 
            
        wdFormatPDF = 17

        in_file = os.path.abspath('Resume.docx')
        out_file = os.path.abspath('Resume.pdf')

        word = client.DispatchEx("Word.Application")
        doc = word.Documents.Open(in_file)
        doc.SaveAs(out_file, FileFormat=wdFormatPDF)
        doc.Close()
        word.Quit()
        
        
        filename_1 = "Resume.pdf"
        attachment_1 = open("Resume.pdf", "rb") 

        # instance of MIMEBase and named as p 
        p_1 = MIMEBase('application', 'octet-stream') 

        # To change the payload into encoded form 
        p_1.set_payload((attachment_1).read()) 

        # encode into base64 
        encoders.encode_base64(p_1) 

        p_1.add_header('Content-Disposition', "attachment; filename= %s" % filename_1) 

        # attach the instance 'p' to instance 'msg' 
        msg.attach(p_1) 

        # creates SMTP session 
        s = smtplib.SMTP('smtp.gmail.com', 587) 

        # start TLS for security 
        s.starttls() 

        # Authentication 
        s.login(fromaddr, YOUR PASSWORD)

        # Converts the Multipart msg into a string 
        text = msg.as_string() 

        # sending the mail 
        s.sendmail(fromaddr, toaddr, text) 

        # terminating the session 
        s.quit() 
                
    return render_template('1 - Copy.html',error=error,experience_level = experience_level,sectors = sectors,qualifications=qualifications,year_passing = year_passing,experiences = experience)


if __name__ == '__main__':
    app.run(debug = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2019 15:56:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2019 15:57:17] "POST / HTTP/1.1" 200 -
